Import packages

In [1]:
import sys, os
import pathlib
__location__ = os.getcwd()
__location__ = pathlib.Path(__location__)

sys.path.append('C:\\Users\\doore\\project\\snake_RL\\dmc\\domains')
import snake
from snake.envs.SnakeEnv import SnakeEnv # for ray env register
import gymnasium as gym
import ray

# import rl 알고리즘
from ray.rllib.algorithms.ppo import PPOConfig 
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig
from ray.rllib.policy.policy import Policy

from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

import numpy as np

c:\Users\doore\anaconda3\envs\rllib\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Make Snake Env

In [2]:
env = gym.make('snake/SnakeEnv-v1', render_mode="human")

Load pre-learned policy

In [6]:
policy = Policy.from_checkpoint('C:/Users/doore/ray_results/PPO_snake_2023-03-09_21-45-50djfsrxig/checkpoint_003426')

Do simulation

In [7]:
_obs, _ = env.reset()

for epi in range(10):
    for i in range(400):
        _act = policy['default_policy'].compute_single_action(_obs)
        _obs, _reward, _, _, _ = env.step(_act[0])
    _obs, _ = env.reset()
    
env.close()